##### Alberto garcía Doménech & Eric Heresi Medina

# Estudio de las colaboraciones en la industria musical

In [7]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import collections as col
#Utilizando community detection de python-louvain
import community as com

In [2]:
# API de spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="33eca6309d3344028987abab11e234bc",
                                                           client_secret="25f6177ad5db47b687266bfdd2e990e2"))

## Charts de Billboard

In [4]:
hot100 = pd.read_excel("Hot 100 Audio Features.xlsx")

In [5]:
import requests
import spotipy.util as sputil
import json

In [6]:
SEARCH_PLAYLIST_ENDPOINT ='https://api.spotify.com/v1/search?type=playlist'

def search_playlist(name):
    path = 'token.json'
    with open(path) as t:
        token = json.load(t)
    myparams = {'type': 'playlilst'}
    myparams['q'] = name
    resp = requests.get(SEARCH_PLAYLIST_ENDPOINT, params=myparams, headers={"Authorization": "Bearer {}".format(token)})
    pprint (resp.json())
    return resp.json()


def query_playlist(playlist):

    path = 'token.json'
    with open(path) as t:
        token = json.load(t)

    sp = spotipy.Spotify(auth=token)

    data = search_playlist(playlist)
    # get playlists
    playlists = data['playlists']['items']

    for playlist in playlists:
        uri = playlist['uri']
        username = uri.split(':')[2]
        playlist_id = uri.split(':')[4]
        # query playlists
        results = sp.user_playlist(username, playlist_id)
        # separate data by track
        tracks = results['tracks']           
        # lookup       
        for item in tracks['items']:
            track = item['track']
            artist = track['artists'][0]['name']
            print(artist)

In [10]:
# BUSCAR LOS ARTISTAS Y FORMAR GRAFOS
def formarGrafo(dataframe, _id,name):
    G = nx.Graph()
    errores = 0
    canciones_totales = 0
    colabos = 0
    for index, row in dataframe.iterrows():
        res = str(row[_id])
        if len(res) > 3:
            canciones_totales += 1
            try:
                result = sp.track(res)
            except:
                errores +=1
            else:
                artists = result["artists"]
                index = 0
                if(len(artists) > 1):
                    colabos += 1
                while index < len(artists) -1:
                    indexb = index+1
                    while indexb < len(artists):
                        #G.add_edge(artists[index]["name"], artists[indexb]["name"],Song = str(row[name]))
                        if G.has_edge(artists[index]["name"], artists[indexb]["name"]):
                            # we added this one before, just increase the weight by one
                            G[artists[index]["name"]][artists[indexb]["name"]]['weight'] += 1
                        else:
                            # new edge. add with weight=1
                            G.add_edge(artists[index]["name"], artists[indexb]["name"], weight=1, Song = str(row[name]))   
                            nx.set_node_attributes(G, values = {artists[index]["name"]:artists[index]["id"], artists[indexb]["name"]:artists[indexb]["id"]}, name='artist_id')
                        indexb+=1
                    index+=1
    return G,errores, canciones_totales, colabos

In [8]:
hot100.head()

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],3jbT1Y5MoPwEIpZndDDwVq,NaN,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,...,10.0,-4.803,0.0,0.0735,0.01700,0.000016,0.1790,0.623,80.002,4.0
1,The Ones That Didn't Make It Back HomeJustin M...,Justin Moore,The Ones That Didn't Make It Back Home,"['arkansas country', 'contemporary country', '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",2VxeLyX666F8uXCJ0dZF8B,NaN,A Star Is Born Soundtrack,0.0,215733.0,88.0,...,7.0,-6.362,1.0,0.0308,0.37100,0.000000,0.2310,0.323,95.799,4.0
3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,"['dfw rap', 'melodic rap', 'rap']",0Xek5rqai2jcOWCYWJfVCF,NaN,Hollywood's Bleeding,1.0,196760.0,86.0,...,6.0,-4.169,1.0,0.2100,0.05880,0.000000,0.0955,0.667,76.388,4.0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"['dfw rap', 'rap', 'southern hip hop', 'trap']",2biNa12dMbHJrHVFRt8JyO,https://p.scdn.co/mp3-preview/fa6fa6f6f363be29...,Bacc At It Again,1.0,228185.0,61.0,...,8.0,-5.725,0.0,0.1680,0.00124,0.000001,0.0716,0.856,135.979,4.0


In [11]:
G_billboard, errores, canciones_totales, colabos = formarGrafo(hot100,"spotify_track_id","Song")

In [12]:
errores

0

In [13]:
print("De ",canciones_totales," canciones " ,colabos, "son colaboraciones")

De  23743  canciones  2868 son colaboraciones


In [14]:
len(G_billboard.nodes())

2338

In [15]:
len(G_billboard.edges())

5102

In [ ]:
sorted(G_billboard.degree, key=lambda x: x[1], reverse=True)

In [ ]:
nx.draw(G_billboard)
plt.show()

In [17]:
# guardamos el grafo para estudiarlo en gephi
nx.write_gexf(G_billboard,"billboard2.gexf")

#### Billboard a traves del tiempo

In [ ]:
# Vamos a crear 3 redes para estudiar la evolucion de las colaboraciones, una desde 1958(inicio de las listas de billboard) hasta 1980,
# otra desde 1980 hasta el 2000 y desde el 2000 hasta la actualidad.


G60 = nx.Graph()
G80 = nx.Graph()
G00 = nx.Graph()


canciones60, canciones80,canciones00 = 0,0,0
colabos60,colabos80,colabos00 = 0,0,0
errores = 0
for index, row in hot100.iterrows():
    res = str(row["spotify_track_id"])
    if len(res) > 3:
        try:
            result = sp.track(res)
        except:
            errores +=1
        else:
            date = result["album"]['release_date']
            artists = result["artists"]
            index = 0
            if(date < '1980-01-01'):
                canciones60+=1
                colabos60 += 1 if (len(artists)>1) else 0
                while index < len(artists) -1:
                    indexb = index+1
                    while indexb < len(artists):
                        #G.add_edge(artists[index]["name"], artists[indexb]["name"],Song = str(row[name]))
                        if G60.has_edge(artists[index]["name"], artists[indexb]["name"]):
                            # we added this one before, just increase the weight by one
                            G60[artists[index]["name"]][artists[indexb]["name"]]['weight'] += 1
                        else:
                            # new edge. add with weight=1
                            G60.add_edge(artists[index]["name"], artists[indexb]["name"], weight=1, Song = str(row["Song"]))   
                            nx.set_node_attributes(G60, values = {artists[index]["name"]:artists[index]["id"], artists[indexb]["name"]:artists[indexb]["id"]}, name='artist_id')
                        indexb+=1
                    index+=1  
            elif(date < '2000-01-01'):
                canciones80+=1
                colabos80+= 1 if (len(artists)>1) else 0
                while index < len(artists) -1:
                    indexb = index+1
                    while indexb < len(artists):
                        #G.add_edge(artists[index]["name"], artists[indexb]["name"],Song = str(row[name]))
                        if G80.has_edge(artists[index]["name"], artists[indexb]["name"]):
                            # we added this one before, just increase the weight by one
                            G80[artists[index]["name"]][artists[indexb]["name"]]['weight'] += 1
                        else:
                            # new edge. add with weight=1
                            G80.add_edge(artists[index]["name"], artists[indexb]["name"], weight=1, Song = str(row["Song"]))   
                            nx.set_node_attributes(G80, values = {artists[index]["name"]:artists[index]["id"], artists[indexb]["name"]:artists[indexb]["id"]}, name='artist_id')
                        indexb+=1
                    index+=1  
            else:
                canciones00 += 1
                colabos00 += 1 if(len(artists)>1) else 0
                while index < len(artists) -1:
                    indexb = index+1
                    while indexb < len(artists):
                        #G.add_edge(artists[index]["name"], artists[indexb]["name"],Song = str(row[name]))
                        if G00.has_edge(artists[index]["name"], artists[indexb]["name"]):
                            # we added this one before, just increase the weight by one
                            G00[artists[index]["name"]][artists[indexb]["name"]]['weight'] += 1
                        else:
                            # new edge. add with weight=1
                            G00.add_edge(artists[index]["name"], artists[indexb]["name"], weight=1, Song = str(row["Song"]))   
                            nx.set_node_attributes(G00, values = {artists[index]["name"]:artists[index]["id"], artists[indexb]["name"]:artists[indexb]["id"]}, name='artist_id')
                        indexb+=1
                    index+=1  

In [ ]:
print("Canciones totales en el Hot100 entre 1958 y 1980: ", canciones60, " de las cuales colaboraciones: ", colabos60)
print("Artistas que colaboraron entre 1958 y 1980 y estuvieron en el top 100: ", len(G60.nodes()))
print("Canciones totales en el Hot100 entre 1980 y 2000: ", canciones80, " de las cuales colaboraciones: ", colabos80)
print("Artistas que colaboraron entre 1980 y 2000 y estuvieron en el top 100: ", len(G80.nodes()))
print("Canciones totales en el Hot100 entre 2000 y la actualidad: ", canciones00, " de las cuales colaboraciones: ", colabos00)
print("Artistas que colaboraron entre 2000 y la actualidad y estuvieron en el top 100: ", len(G00.nodes()))

In [ ]:
# Vemos una evolución clara en el número de colaboraciones
# Exportamos las redes para estudiarlas a fondo en gephi

nx.write_gexf(G60,"billboard60_1.gexf")
nx.write_gexf(G80,"billboard80_1.gexf")
nx.write_gexf(G00,"billboard00_1.gexf")


In [12]:
# funcion para conseguir id de un artista
def get_id(name):
    x = ""
    i = 0
    sp.search(name)['tracks']['items'][1]['album']['artists']
    while(x != name and i < len(sp.search(name)['tracks']['items'])):
        artists = sp.search(name)['tracks']['items'][i]['album']['artists']
        for a in artists:
            if(a['name'] == name):
                x=name
                id = a['id']
            i+=1
    return id
# funcion para mostrar los generos de los x artistas con mas colaboraciones
# algunos artistas no tienen generos relacionados por lo que aparecera una lista vacia[]
def get_generos(G,x):
    for i in range(0,x):
        try:
            name = sorted(G.degree, key=lambda x: x[1], reverse=True)[i][0]
            print("Artista: ",name, " generos: ", sp.artist(get_id(name))['genres'] )
        except:
            print("fallo al buscar artista ", name)

In [13]:
# Vamos a ver los 10 artistas con más colaboraciones en el top 100 entre 1958 y 1980
for i in range(0,10):
    print(sorted(G60.degree, key=lambda x: x[1], reverse=True)[i])

('Chaka Khan', 5)
('Marvin Gaye', 4)
('Quincy Jones', 4)
('Dinah Washington', 3)
('Nickolas Ashford', 3)
('Valerie Simpson', 3)
('Diana Ross', 2)
('Carly Simon', 2)
('Johnny Mathis', 2)
('Ashford & Simpson', 2)


In [14]:
# Obtenemos los id de los artistas con mas colaboraciones para ver con que generos estan relacionados

# Excepto el 5 artista, Nickolas Ashford, los demas los conseguimos mediante la api, Nickolas Ashford al no haber publicado ninguna
# canción y solo haber colaborado tenemos que obtener su id de forma manual (4X7jBy7ujZ6PSsHJPuCosZ)
# Al no haber publicado ninguna canción por si mismo, Nickolas Ashford no tiene ningún genero asociado pero como sus colaboraciones
# son con Quincy Jones asumimos que comparten los generos 

get_generos(G60,10)

Artista:  Chaka Khan  generos:  ['dance pop', 'disco', 'funk', 'motown', 'neo soul', 'post-disco', 'quiet storm', 'soul', 'urban contemporary']
Artista:  Marvin Gaye  generos:  ['classic soul', 'funk', 'motown', 'northern soul', 'quiet storm', 'soul']
Artista:  Quincy Jones  generos:  ['adult standards', 'disco', 'funk', 'jazz funk', 'motown', 'quiet storm', 'soul']
Artista:  Dinah Washington  generos:  ['adult standards', 'jazz blues', 'lounge', 'soul', 'swing', 'vocal jazz']
fallo al buscar artista  Nickolas Ashford
Artista:  Valerie Simpson  generos:  []
Artista:  Diana Ross  generos:  ['adult standards', 'disco', 'funk', 'mellow gold', 'motown', 'quiet storm', 'soft rock', 'soul']
Artista:  Carly Simon  generos:  ['adult standards', 'brill building pop', 'classic rock', 'classic uk pop', 'country rock', 'folk', 'folk rock', 'mellow gold', 'soft rock']
Artista:  Johnny Mathis  generos:  ['adult standards', 'brill building pop', 'easy listening', 'lounge', 'mellow gold', 'vocal jazz'

In [15]:
# Vamos a ver los 10 artistas con más colaboraciones en el top 100 entre 1980 y 2000
for i in range(0,10):
    print(sorted(G80.degree, key=lambda x: x[1], reverse=True)[i])

('Babyface', 15)
('JAY-Z', 13)
('Jody Watley', 12)
('Grover Washington, Jr.', 10)
('Quincy Jones', 9)
('Rod Stewart', 9)
('Tamia', 9)
('The Blackout Allstars', 8)
('Ray Barretto', 8)
('Sheila E.', 8)


In [16]:
# Obtenemos los id de los artistas con mas colaboraciones para ver con que generos estan relacionados
# El grupo The Blackout Allstars no tiene perfil de spotify por lo que la busqueda de generos falla. Es un supergrupo formado por 8 artistas latinos (de ahi sus 8 colaboraciones)
get_generos(G80,10)

Artista:  Babyface  generos:  ['dance pop', 'neo soul', 'new jack swing', 'quiet storm', 'r&b', 'urban contemporary']
Artista:  JAY-Z  generos:  ['east coast hip hop', 'hip hop', 'pop rap', 'rap']
Artista:  Jody Watley  generos:  ['disco', 'freestyle', 'minneapolis sound', 'new jack swing', 'new wave pop', 'post-disco', 'quiet storm', 'urban contemporary']
Artista:  Grover Washington, Jr.  generos:  []
Artista:  Quincy Jones  generos:  ['adult standards', 'disco', 'funk', 'jazz funk', 'motown', 'quiet storm', 'soul']
Artista:  Rod Stewart  generos:  ['mellow gold', 'soft rock']
Artista:  Tamia  generos:  ['canadian pop', 'dance pop', 'hip pop', 'neo soul', 'new jack swing', 'quiet storm', 'r&b', 'urban contemporary']
fallo al buscar artista  The Blackout Allstars
Artista:  Ray Barretto  generos:  ['boogaloo', 'latin jazz', 'salsa', 'salsa international', 'tropical']
Artista:  Sheila E.  generos:  ['freestyle', 'funk', 'minneapolis sound', 'post-disco', 'quiet storm', 'rock drums', 'urb

In [17]:
# Vamos a ver los 10 artistas con más colaboraciones en el top 100 entre 1980 y 2000
for i in range(0,10):
    print(sorted(G00.degree, key=lambda x: x[1], reverse=True)[i])

('Lil Wayne', 89)
('Drake', 66)
('Nicki Minaj', 66)
('Chris Brown', 65)
('Kanye West', 63)
('Fat Joe', 53)
('Future', 50)
('T-Pain', 49)
('Ludacris', 49)
('DJ Khaled', 45)


In [18]:
# Obtenemos los id de los artistas con mas colaboraciones para ver con que generos estan relacionados
get_generos(G00,10)

Artista:  Lil Wayne  generos:  ['hip hop', 'new orleans rap', 'pop rap', 'rap', 'trap']
Artista:  Drake  generos:  ['canadian hip hop', 'canadian pop', 'hip hop', 'pop rap', 'rap', 'toronto rap']
Artista:  Nicki Minaj  generos:  ['dance pop', 'hip pop', 'pop', 'pop dance', 'pop rap', 'post-teen pop', 'queens hip hop']
Artista:  Chris Brown  generos:  ['dance pop', 'pop', 'pop rap', 'r&b', 'rap']
Artista:  Kanye West  generos:  ['chicago rap', 'rap']
Artista:  Fat Joe  generos:  ['bronx hip hop', 'dance pop', 'east coast hip hop', 'gangster rap', 'hardcore hip hop', 'hip hop', 'hip pop', 'pop rap', 'rap', 'trap']
Artista:  Future  generos:  ['atl hip hop', 'pop rap', 'rap', 'southern hip hop', 'trap']
Artista:  T-Pain  generos:  ['dance pop', 'hip hop', 'pop', 'pop rap', 'r&b', 'rap', 'southern hip hop', 'trap', 'urban contemporary']
Artista:  Ludacris  generos:  ['atl hip hop', 'dance pop', 'dirty south rap', 'hip hop', 'pop rap', 'rap', 'southern hip hop', 'trap']
Artista:  DJ Khaled 

## Charts de Spotify

In [15]:
from fycharts.SpotifyCharts import SpotifyCharts

api = SpotifyCharts()
# api.top200Daily(output_file = "top_200_daily.csv",region="global")
api.top200Weekly(output_file="GLOBAL_top_200_weekly.csv", start = "2016-12-23", region = "global")
api.top200Weekly(output_file="SPAIN_top_200_weekly.csv",start = "2016-12-23", region = "es")


INFO : 20/01/2021 07:24:50 PM : Extracting top 200 weekly for 2016-12-23--2016-12-30 - global
INFO : 20/01/2021 07:24:51 PM : Extracting top 200 weekly for 2016-12-30--2017-01-06 - global
INFO : 20/01/2021 07:24:51 PM : Appending data to the file GLOBAL_top_200_weekly.csv...
INFO : 20/01/2021 07:24:51 PM : Done appending to the file GLOBAL_top_200_weekly.csv!!!
INFO : 20/01/2021 07:24:52 PM : Extracting top 200 weekly for 2017-01-06--2017-01-13 - global
INFO : 20/01/2021 07:24:52 PM : Appending data to the file GLOBAL_top_200_weekly.csv...
INFO : 20/01/2021 07:24:52 PM : Done appending to the file GLOBAL_top_200_weekly.csv!!!
INFO : 20/01/2021 07:24:54 PM : Extracting top 200 weekly for 2017-01-13--2017-01-20 - global
INFO : 20/01/2021 07:24:54 PM : Appending data to the file GLOBAL_top_200_weekly.csv...
INFO : 20/01/2021 07:24:54 PM : Done appending to the file GLOBAL_top_200_weekly.csv!!!
INFO : 20/01/2021 07:24:55 PM : Extracting top 200 weekly for 2017-01-20--2017-01-27 - global
IN

In [27]:
# G_Global_weekly = nx.Graph()
globalWeekly = pd.read_csv("GLOBAL_top_200_weekly.csv")
globalWeekly.head()

,Position,Track Name,Artist,Streams,date,region,spotify_id
0,1,Starboy,The Weeknd,25286465,2016-12-23--2016-12-30,global,5aAx2yezTd8zXrkmtKl66Z
1,2,Closer,The Chainsmokers,22047697,2016-12-23--2016-12-30,global,7BKLCZ1jbUBVqRi2FVlTVw
2,3,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,19794482,2016-12-23--2016-12-30,global,5knuzwU65gJK7IF5yJsuaW
3,4,Let Me Love You,DJ Snake,17965723,2016-12-23--2016-12-30,global,4pdPtRcBmOSQDlJ3Fk945m
4,5,Don't Wanna Know,Maroon 5,16966668,2016-12-23--2016-12-30,global,5MFzQMkrl1FOOng9tq6R9r


In [ ]:
G_Global_weekly, errores = formarGrafo(globalWeekly,"spotify_id", "Track Name")

In [1]:
errores

NameError: name 'errores' is not defined

In [ ]:
len(G_Global_weekly.nodes())

In [ ]:
len(G_Global_weekly.edges())

In [ ]:
sorted(G_Global_weekly.degree, key=lambda x: x[1], reverse=True)

In [ ]:
# guardamos el grafo para estudiarlo en gephi
nx.write_gexf(G_Global_weekly,"spotifyGlobalWeekly1.gexf")

In [ ]:
G_Spain = nx.Graph()
spain_weekly = pd.read_csv("SPAIN_top_200_weekly.csv")
spain_weekly.head()

In [ ]:
G_Spain, errores = formarGrafo(G_Spain,globalWeekly,"spotify_id", "Track Name")

In [ ]:
errores

In [ ]:
len(G_Spain.nodes())

In [ ]:
len(G_Spain.edges())

In [ ]:
sorted(G_Spain.degree, key=lambda x: x[1], reverse=True)

In [ ]:
# guardamos el grafo para estudiarlo en gephi
nx.write_gexf(G_Spain,"spotifySpainWeekly1.gexf")